In [1]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 11.2 MB/s eta 0:00:00


In [3]:
from google.colab import files
uploading = files.upload()

Saving resume_data.zip to resume_data.zip


In [4]:
!pip install patool
import patoolib
patoolib.extract_archive("/content/resume_data.zip")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.0 MB/s eta 0:00:00


INFO patool: Extracting /content/resume_data.zip ...
INFO:patool:Extracting /content/resume_data.zip ...
INFO patool: running /usr/bin/7z x -o./Unpack_q8n5v23h -- /content/resume_data.zip
INFO:patool:running /usr/bin/7z x -o./Unpack_q8n5v23h -- /content/resume_data.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... /content/resume_data.zip extracted to `data'.
INFO:patool:... /content/resume_data.zip extracted to `data'.


'data'

In [7]:
!pip install python-docx
!pip install pytesseract
!pip install Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.6 MB/s eta 0:00:00


In [9]:
import os
import re
import fitz  # PyMuPDF
import docx
from PIL import Image
import pytesseract
from difflib import SequenceMatcher

def convert_image_to_text(image_path):
    """Convert image to text using Tesseract OCR."""
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def convert_pdf_to_text(pdf_path):
    """Convert PDF to text using PyMuPDF and Tesseract OCR for images."""
    pdf_document = fitz.open(pdf_path)
    text = ""

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        # Try to extract text using PyMuPDF
        page_text = page.get_text()
        if page_text.strip():
            text += page_text
        else:
            # Fall back to OCR if text extraction fails
            pix = page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            page_text = pytesseract.image_to_string(img)
            text += page_text

    return text

resume_text = (convert_pdf_to_text("/content/data/Pradip Kumar Verma _1716878268725-1.pdf"))


In [5]:
def extract_details_from_text(text):
    prompt = (
        f"Please extract the following details from the given resume text:\n\n"
        f"{text}\n\n"
        "Format your response exactly as follows:\n"
        "Name: ...\n"
        "Contact: ...\n"
        "Email: ...\n"
        "Education: ...\n"
        "Experience: ...\n"
    )
    response = conversation_pipeline(prompt, max_length=256, clean_up_tokenization_spaces=True)[0]['generated_text']
    return response


In [17]:
print(resume_text)

Pradip Kumar Verma
9998296042
vpradeep359@gmail.com
OBJECTIVE
Looking for a challenging role so that I can use my skills and capabilities through sincere
dedication and hard work for successful career.
SKILLS
Communication leadership problem solving Active listening Teamwork Customer
service
EXPERIENCE
23/11/2015 - 08/04/2019
Tata Starbucks
Barista
Making coffee cash handle inventory
25/04/2019 - 01/11/2023
Hotel Las Vegas cafe bae
Senior barista
Operations support
Beverage innovation
Coffee training
14/11/2023 -
Compass group India
Jr executive
Operations support
Team handle
Duty allocation
Beverage innovation
Coffee training
PROJECTS
EDUCATION
Degree / Course
University / Board
Percentage /
CGPA
Year of passing
10
Jai Jind inter college tezi
bajar jaunpur
55
2010
12
Jai hind inter college tezi
Bazar jaunpur
65
2012
Graduation
Purvanchal university
55
2016
ADDITIONAL PERSONAL INFO
Address
House number 53 patel chest Hudson lane gtb Nagar
delhi
Languages
Hindi & English
Date of Birth
2

In [10]:
from transformers import pipeline

# Load the conversational model from Hugging Face
model_name = "google/flan-t5-small"
conversation_pipeline = pipeline("text2text-generation", model=model_name)

def extract_details_from_text(text):
    prompt = (
        f"Please extract the following details from the given resume text:\n\n"
        f"{text}\n\n"
        "Format your response exactly as follows:\n"
        "Name: ...\n"
        "Contact: ...\n"
        "Email: ...\n"
        "Education: ...\n"
        "Experience: ...\n"
    )
    response = conversation_pipeline(prompt, max_length=256, clean_up_tokenization_spaces=True)[0]['generated_text']
    return response
# resume_details = [extract_details_from_text(text) for text in resume_texts]
resume_details = extract_details_from_text(resume_text)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [16]:
print(resume_details)

- e-mail address - name - e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-e-


In [ ]:
# import re

# def parse_resume_details(details_text):
#     details = {
#         "Name": re.search(r"Name:\s*(.+)", details_text),
#         "Contact": re.search(r"Contact:\s*(.+)", details_text),
#         "Email": re.search(r"Email:\s*(.+)", details_text),
#         "Education": re.search(r"Education:\s*(.+)", details_text),
#         "Experience": re.search(r"Experience:\s*(.+)", details_text)
#     }
#     # Use group(1) if the match is found, otherwise return None
#     parsed_details = {key: (match.group(1).strip() if match else None) for key, match in details.items()}
#     return parsed_details

# parsed_resumes = [parse_resume_details(details) for details in resume_details]


In [13]:
import re

def parse_resume_details(details_text):
    details = {
        "Name": re.search(r"Name: (.+)", details_text).group(1) if re.search(r"Name: (.+)", details_text) else None,
        "Contact": re.search(r"Contact: (.+)", details_text).group(1) if re.search(r"Contact: (.+)", details_text) else None,
        "Email": re.search(r"Email: (.+)", details_text).group(1) if re.search(r"Email: (.+)", details_text) else None,
        "Education": re.search(r"Education: (.+)", details_text).group(1) if re.search(r"Education: (.+)", details_text) else None,
        "Experience": re.search(r"Experience: (.+)", details_text).group(1) if re.search(r"Experience: (.+)", details_text) else None
    }
    return details

# parsed_resumes = [parse_resume_details(details) for details in resume_details]
parsed_resumes = parse_resume_details(resume_details)




In [15]:
print(parsed_resumes)

{'Name': None, 'Contact': None, 'Email': None, 'Education': None, 'Experience': None}


In [14]:
import csv

with open('parsed_resumes.csv', 'w', newline='') as csvfile:
    fieldnames = ['Name', 'Contact', 'Email', 'Education', 'Experience']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for resume in parsed_resumes:
        writer.writerow(resume)


AttributeError: 'str' object has no attribute 'keys'